In [1]:
import sys
import os
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if path not in sys.path:
    sys.path.insert(0, path)

In [2]:
from omegaconf import OmegaConf
import pyvista as pv
import torch
import numpy as np

In [3]:
# visualization configuration
os.environ["DISPLAY"] = ":1.0"
os.environ["PYVISTA_OFF_SCREEN"]="true"
os.environ["PYVISTA_PLOT_THEME"]="true"
os.environ["PYVISTA_USE_PANEL"]="true"
os.environ["PYVISTA_AUTO_CLOSE"]="false"
os.system("Xvfb :1 -screen 0 1024x768x24 > /dev/null 2>&1 &")

0

## Check your raw data

In [4]:
import json
import laspy

raw_path = 'data/rocklas/raw'
for i, split in enumerate(["train", "valid", "test"]):
    json_path = os.path.join(raw_path, f"{split}_split.json")
    with open(json_path, "r") as f:
        filenames = json.load(f)
    las_files = [os.path.join(raw_path, f) for f in filenames]
    for las_file in las_files:
        las = laspy.read(las_file)

In [5]:
for i in las.point_format:
    print(i.name)
    
# points with real local coordinates
las.x.scaled_array().min()
las.x.scaled_array().max()
las.y.scaled_array().min()
las.y.scaled_array().max()
las.z.scaled_array().min()
las.z.scaled_array().max()

# color value has type of uint16
print(las.red.max())
print(las.red.min())
print(las.green.max())
print(las.green.min())
print(las.blue.max())
print(las.blue.min())


X
Y
Z
intensity
return_number
number_of_returns
scan_direction_flag
edge_of_flight_line
classification
synthetic
key_point
withheld
scan_angle_rank
user_data
point_source_id
red
green
blue
notPBR
Original cloud index
isPBR
65280
0
65280
0
62464
0


## Test dataloading script

In [4]:
DIR = "" # Replace with your root directory, the data will go in DIR/data.
USE_COLOR = True #@param {type:"boolean"}

In [52]:
pbr_yaml = """
dataset_name: rock_las                         # dataset name
dataroot: %s
color: %r                                     # Use color vectors as features
class: None # shapenet.ShapeNetDataset
task: segmentation
first_subsampling: 0.02                      # Grid size of the input data
pre_transforms:                               # Offline transforms, done only once         
    - transform: GridSampling3D
      params:
        size: ${first_subsampling}
train_transforms:                             # Data augmentation pipeline
    - transform: RandomNoise
      params:
        sigma: 0.01
        clip: 0.05
    - transform: RandomScaleAnisotropic
      params:
        scales: [0.9,1.1]
    - transform: AddOnes
    - transform: AddFeatsByKeys
      params:
        list_add_to_x: [True]
        feat_names: ["ones"]
        delete_feats: [True]
test_transforms:
    - transform: AddOnes
    - transform: AddFeatsByKeys
      params:
        list_add_to_x: [True]
        feat_names: ["ones"]
        delete_feats: [True]
""" % (os.path.join(DIR,"data"), USE_COLOR) 


from omegaconf import OmegaConf
params = OmegaConf.create(pbr_yaml)


In [53]:
from rock_detection_3d.datasets.segmentation.rock_las import RockLASDataset

In [54]:
dataset = RockLASDataset(params)
dataset

Dataset: RockLASDataset 
train_pre_batch_collate_transform = None
val_pre_batch_collate_transform = None
test_pre_batch_collate_transform = None
pre_transform = Compose([
    GridSampling3D(grid_size=0.02, quantize_coords=False, mode=mean),
])
test_transform = Compose([
    AddOnes(),
    AddFeatsByKeys(ones=True),
])
train_transform = Compose([
    RandomNoise(sigma=0.01, clip=0.05),
    RandomScaleAnisotropic([0.9, 1.1]),
    AddOnes(),
    AddFeatsByKeys(ones=True),
])
val_transform = None
inference_transform = Compose([
    GridSampling3D(grid_size=0.02, quantize_coords=False, mode=mean),
    AddOnes(),
    AddFeatsByKeys(ones=True),
])
Size of train_dataset = 29
Size of test_dataset = 10
Size of val_dataset = 10
Batch size = None

In [56]:
#@title Plot samples with part annotations { run: "auto" }
objectid_1 = 0 #@param {type:"slider", min:0, max:100, step:1}
objectid_2 = 1 #@param {type:"slider", min:0, max:100, step:1}
objectid_3 = 2 #@param {type:"slider", min:0, max:100, step:1}

samples = [objectid_1,objectid_2,objectid_3]
p = pv.Plotter(notebook=True,shape=(1, len(samples)),window_size=[1024,412])
for i in range(len(samples)):
    p.subplot(0, i)
    sample = dataset.train_dataset[samples[i]]
    point_cloud = pv.PolyData(sample.pos.numpy())
    point_cloud['y'] = sample.y.numpy()
    p.add_points(point_cloud,  show_scalar_bar=False, point_size=4)
    p.camera_position = [-1,5, -10]
p.show()

ViewInteractiveWidget(height=412, layout=Layout(height='auto', width='100%'), width=1024)

In [11]:
#@title Plot samples with part annotations { run: "auto" }
objectid_1 = 6 #@param {type:"slider", min:0, max:100, step:1}
objectid_2 = 7 #@param {type:"slider", min:0, max:100, step:1}
objectid_3 = 8 #@param {type:"slider", min:0, max:100, step:1}

samples = [objectid_1,objectid_2,objectid_3]
p = pv.Plotter(notebook=True,shape=(1, len(samples)),window_size=[1024,412])
for i in range(len(samples)):
    p.subplot(0, i)
    sample = dataset.test_dataset[0][samples[i]]  # somehow, test_dataset is a list
    point_cloud = pv.PolyData(sample.pos.numpy())
    point_cloud['y'] = sample.y.numpy()
    p.add_points(point_cloud,  show_scalar_bar=False, point_size=4)
    p.camera_position = [-1,5, -10]
p.show()

ViewInteractiveWidget(height=412, layout=Layout(height='auto', width='100%'), width=1024)

In [12]:
print(dataset.train_dataset[1])
las_file = os.path.join(raw_path, 'pbr12.las')
las = laspy.read(las_file)
print(las)

Data(category=[9111], center=[3], file_name="pbr122.las", grid_size=[1], id_scan=[1], origin_id=[9111], pos=[9111, 3], scale=[1], x=[9111, 4], y=[9111])
<LasData(1.2, point fmt: <PointFormat(2, 24 bytes of extra dims)>, 2677 points, 1 vlrs)>


In [17]:
dataset.test_dataset[0][0]

Data(category=[12384], center=[3], file_name="pbr116.las", grid_size=[1], id_scan=[1], origin_id=[12384], pos=[12384, 3], scale=[1], x=[12384, 4], y=[12384])

In [14]:
dataset.train_dataset[0].pos

tensor([[ 1.0750,  0.7057, -0.7976],
        [ 1.0451,  0.6248, -0.7792],
        [ 1.0211,  0.6682, -0.7754],
        ...,
        [-1.0732, -0.7998,  0.5772],
        [-1.0957, -0.7811,  0.5852],
        [-1.0892, -0.7882,  0.5893]])

In [15]:
dataset.train_dataset[0].pos

tensor([[ 0.9939,  0.8073, -0.9157],
        [ 0.9698,  0.6967, -0.8897],
        [ 0.9390,  0.7313, -0.8854],
        ...,
        [-0.9963, -0.9256,  0.6848],
        [-1.0076, -0.8618,  0.6913],
        [-1.0003, -0.8856,  0.6789]])

In [18]:
dataset.val_dataset[0]

Data(category=[21475], center=[3], file_name="pbr10.las", grid_size=[1], id_scan=[1], origin_id=[21475], pos=[21475, 3], scale=[1], x=[21475, 4], y=[21475])